In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [79]:
var = "tg_mean"

In [3]:
#Variable 
variable85="rcp85_tg_mean_seasonal"
variable45="rcp45_tg_mean_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_seasonal.nc']

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [46]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    #print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 4 dataframes

In [66]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C["tg_mean_p10"] = round(df45C["tg_mean_p10"] -273.15,2)
    df45C["tg_mean_p50"] = round(df45C["tg_mean_p50"] -273.15,2)
    df45C["tg_mean_p90"] = round(df45C["tg_mean_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df45f.columns = [season + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [84]:
#df45list


In [82]:
#df45list[0]

In [69]:
dfw45 = df45list[0].reset_index()
dfsp45 = df45list[1].reset_index()
dfsu45 = df45list[2].reset_index()
dff45 = df45list[3].reset_index()

In [74]:
mkey = ["lat","lon"]

In [75]:
dfse45 = dfw45.merge(dfsp45, on=mkey).merge(dfsu45, on=mkey).merge(dff45, on=mkey)

In [83]:
#dfse45

In [80]:
dfse45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_seasonal.feather")

In [81]:
dfse452 = dfse45.drop(columns=["winter_hist_rcp45_p10","winter_hist_rcp45_p50", "winter_hist_rcp45_p90", 
                               "spring_hist_rcp45_p10", "spring_hist_rcp45_p50", "spring_hist_rcp45_p90", 
                               "summer_hist_rcp45_p10","summer_hist_rcp45_p50", "summer_hist_rcp45_p90", 
                               "fall_hist_rcp45_p10","fall_hist_rcp45_p50", "fall_hist_rcp45_p90"])
dfse452


,lat,lon,winter_t2050_rcp45_p10,winter_t2080_rcp45_p10,winter_t2050_rcp45_p50,winter_t2080_rcp45_p50,winter_t2050_rcp45_p90,winter_t2080_rcp45_p90,spring_t2050_rcp45_p10,spring_t2080_rcp45_p10,...,summer_t2050_rcp45_p50,summer_t2080_rcp45_p50,summer_t2050_rcp45_p90,summer_t2080_rcp45_p90,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90
0,40.041039,-89.045212,-1.56,-1.23,0.52,0.95,2.59,3.24,11.54,12.24,...,25.45,26.19,26.90,27.64,13.61,13.90,15.06,15.65,16.62,17.39
1,40.041039,-88.961884,-1.58,-1.25,0.49,0.93,2.57,3.21,11.51,12.21,...,25.44,26.18,26.89,27.62,13.60,13.89,15.06,15.65,16.63,17.39
2,40.041039,-88.878548,-1.59,-1.27,0.48,0.92,2.55,3.19,11.50,12.19,...,25.44,26.18,26.88,27.61,13.61,13.89,15.07,15.66,16.63,17.39
3,40.041039,-88.795219,-1.61,-1.28,0.45,0.90,2.52,3.16,11.47,12.16,...,25.43,26.18,26.88,27.61,13.59,13.88,15.05,15.64,16.61,17.37
4,40.041039,-88.711891,-1.60,-1.27,0.46,0.91,2.53,3.17,11.49,12.18,...,25.46,26.20,26.90,27.62,13.63,13.91,15.08,15.67,16.64,17.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-20.47,-19.23,-17.51,-16.40,-14.07,-13.24,-14.24,-13.68,...,4.51,4.37,6.46,6.09,-6.70,-6.48,-4.47,-4.06,-2.62,-2.02
76202,66.623306,-61.546310,-20.62,-19.40,-17.67,-16.56,-14.25,-13.42,-14.55,-14.00,...,4.16,4.02,6.12,5.75,-7.07,-6.85,-4.86,-4.45,-3.02,-2.42
76203,66.623306,-61.462978,-20.46,-19.24,-17.51,-16.40,-14.08,-13.26,-14.31,-13.76,...,4.38,4.24,6.34,5.98,-6.84,-6.62,-4.63,-4.23,-2.79,-2.19
76204,66.623306,-61.379650,-20.38,-19.18,-17.44,-16.33,-14.02,-13.19,-14.21,-13.66,...,4.44,4.32,6.40,6.07,-6.76,-6.54,-4.56,-4.16,-2.73,-2.13


In [ ]:
dfsew = dfse45[["winter_hist_rcp45_p50"]]
dfsesp = dfse45[["spring_hist_rcp45_p50"]]
dfsesu = dfse45[["summer_hist_rcp45_p50"]]
dfsefa = dfse45[["fall_hist_rcp45_p50"]]

### TO-DO: 
- get the means for historic after rcp85


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [94]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 4 dataframes

In [103]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C["tg_mean_p10"] = round(df85C["tg_mean_p10"] -273.15,2)
    df85C["tg_mean_p50"] = round(df85C["tg_mean_p50"] -273.15,2)
    df85C["tg_mean_p90"] = round(df85C["tg_mean_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df85f.columns = [season + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [104]:
df85list


[                      winter_hist_rcp85_p10  winter_t2050_rcp85_p10  \
 lat       lon                                                         
 40.041039 -89.045212                  -3.91                   -0.84   
           -88.961884                  -3.94                   -0.87   
           -88.878548                  -3.96                   -0.89   
           -88.795219                  -3.98                   -0.91   
           -88.711891                  -3.98                   -0.91   
 ...                                     ...                     ...   
 66.623306 -61.629639                 -25.20                  -18.88   
           -61.546310                 -25.34                  -19.03   
           -61.462978                 -25.19                  -18.87   
           -61.379650                 -25.11                  -18.79   
           -61.296322                 -24.67                  -18.38   
 
                       winter_t2080_rcp85_p10  winter_hist_rcp

In [82]:
#df45list[0]

In [105]:
dfw85 = df85list[0].reset_index()
dfsp85 = df85list[1].reset_index()
dfsu85 = df85list[2].reset_index()
dff85 = df85list[3].reset_index()

In [106]:
mkey = ["lat","lon"]

In [107]:
dfse85 = dfw85.merge(dfsp85, on=mkey).merge(dfsu85, on=mkey).merge(dff85, on=mkey)

In [108]:
dfse85

,lat,lon,winter_hist_rcp85_p10,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_hist_rcp85_p50,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_hist_rcp85_p90,winter_t2050_rcp85_p90,...,summer_t2080_rcp85_p90,fall_hist_rcp85_p10,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_hist_rcp85_p50,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_hist_rcp85_p90,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,-3.91,-0.84,0.95,-1.35,1.13,3.00,0.63,3.29,...,30.51,11.56,14.50,16.33,12.78,16.02,18.18,14.14,17.63,19.91
1,40.041039,-88.961884,-3.94,-0.87,0.94,-1.37,1.11,2.98,0.61,3.26,...,30.49,11.55,14.50,16.33,12.78,16.02,18.18,14.14,17.63,19.91
2,40.041039,-88.878548,-3.96,-0.89,0.92,-1.39,1.10,2.97,0.59,3.25,...,30.48,11.56,14.51,16.33,12.78,16.02,18.18,14.13,17.62,19.90
3,40.041039,-88.795219,-3.98,-0.91,0.91,-1.41,1.07,2.94,0.56,3.22,...,30.47,11.54,14.49,16.31,12.76,16.00,18.16,14.11,17.59,19.88
4,40.041039,-88.711891,-3.98,-0.91,0.92,-1.41,1.08,2.96,0.56,3.22,...,30.48,11.57,14.53,16.34,12.80,16.03,18.19,14.13,17.62,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-25.20,-18.88,-14.48,-22.32,-15.12,-10.85,-19.45,-11.99,...,8.11,-8.33,-5.81,-5.53,-6.39,-3.79,-2.48,-4.46,-1.71,0.57
76202,66.623306,-61.546310,-25.34,-19.03,-14.64,-22.47,-15.29,-11.03,-19.61,-12.17,...,7.77,-8.68,-6.18,-5.90,-6.75,-4.18,-2.88,-4.83,-2.11,0.14
76203,66.623306,-61.462978,-25.19,-18.87,-14.49,-22.32,-15.14,-10.89,-19.45,-12.01,...,8.00,-8.44,-5.95,-5.68,-6.49,-3.95,-2.68,-4.58,-1.88,0.34
76204,66.623306,-61.379650,-25.11,-18.79,-14.42,-22.24,-15.08,-10.85,-19.39,-11.96,...,8.09,-8.34,-5.87,-5.60,-6.39,-3.88,-2.63,-4.49,-1.82,0.38


In [109]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_seasonal.feather")

# Merge df45 and 85

# Extract hist

In [140]:
dfse852 = dfse85.drop(columns=["winter_hist_rcp85_p10","winter_hist_rcp85_p50", "winter_hist_rcp85_p90", 
                               "spring_hist_rcp85_p10", "spring_hist_rcp85_p50", "spring_hist_rcp85_p90", 
                               "summer_hist_rcp85_p10","summer_hist_rcp85_p50", "summer_hist_rcp85_p90", 
                               "fall_hist_rcp85_p10","fall_hist_rcp85_p50", "fall_hist_rcp85_p90"])
dfse852


,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,summer_t2050_rcp85_p50,summer_t2080_rcp85_p50,summer_t2050_rcp85_p90,summer_t2080_rcp85_p90,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,-0.84,0.95,1.13,3.00,3.29,5.56,12.31,13.66,...,26.62,28.73,28.19,30.51,14.50,16.33,16.02,18.18,17.63,19.91
1,40.041039,-88.961884,-0.87,0.94,1.11,2.98,3.26,5.54,12.29,13.63,...,26.60,28.71,28.17,30.49,14.50,16.33,16.02,18.18,17.63,19.91
2,40.041039,-88.878548,-0.89,0.92,1.10,2.97,3.25,5.53,12.27,13.62,...,26.61,28.71,28.16,30.48,14.51,16.33,16.02,18.18,17.62,19.90
3,40.041039,-88.795219,-0.91,0.91,1.07,2.94,3.22,5.50,12.24,13.59,...,26.61,28.71,28.15,30.47,14.49,16.31,16.00,18.16,17.59,19.88
4,40.041039,-88.711891,-0.91,0.92,1.08,2.96,3.22,5.51,12.26,13.61,...,26.63,28.73,28.16,30.48,14.53,16.34,16.03,18.19,17.62,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-18.88,-14.48,-15.12,-10.85,-11.99,-6.04,-13.09,-10.96,...,4.86,5.86,6.96,8.11,-5.81,-5.53,-3.79,-2.48,-1.71,0.57
76202,66.623306,-61.546310,-19.03,-14.64,-15.29,-11.03,-12.17,-6.26,-13.42,-11.29,...,4.51,5.49,6.63,7.77,-6.18,-5.90,-4.18,-2.88,-2.11,0.14
76203,66.623306,-61.462978,-18.87,-14.49,-15.14,-10.89,-12.01,-6.13,-13.18,-11.05,...,4.73,5.72,6.86,8.00,-5.95,-5.68,-3.95,-2.68,-1.88,0.34
76204,66.623306,-61.379650,-18.79,-14.42,-15.08,-10.85,-11.96,-6.11,-13.09,-10.96,...,4.80,5.78,6.93,8.09,-5.87,-5.60,-3.88,-2.63,-1.82,0.38


## Getting historic mean for each season

In [129]:
### Function
def mean_season(season, dfse45, dfse85):
    dfw45 = dfse45[["lat", "lon", season+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", season+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[season+"_hist_p50"] = round((dfsew[season+"_hist_rcp45_p50"]+ dfsew[season+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", season+"_hist_p50"]]
    return (dftgwh3)


### Winter historic mean

In [141]:
dfw45 = dfse45[["lat", "lon", "winter_hist_rcp45_p50"]]
dfw85 = dfse85[["lat", "lon", "winter_hist_rcp85_p50"]]
dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
dfsew["winter_hist_p50"] = round((dfsew["winter_hist_rcp45_p50"]+ dfsew["winter_hist_rcp85_p50"])/2, 2)
dftgwh2 = dfsew.reset_index()
dftgwh3 = dftgwh2[["lat", "lon", "winter_hist_p50"]]
dftgwh3

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,-1.36
1,40.041039,-88.961884,-1.38
2,40.041039,-88.878548,-1.40
3,40.041039,-88.795219,-1.42
4,40.041039,-88.711891,-1.42
...,...,...,...
76201,66.623306,-61.629639,-22.26
76202,66.623306,-61.546310,-22.42
76203,66.623306,-61.462978,-22.26
76204,66.623306,-61.379650,-22.18


In [137]:
dftgwh = mean_season("winter", dfse45, dfse85)
dftgwh

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,-1.36
1,40.041039,-88.961884,-1.38
2,40.041039,-88.878548,-1.40
3,40.041039,-88.795219,-1.42
4,40.041039,-88.711891,-1.42
...,...,...,...
76201,66.623306,-61.629639,-22.26
76202,66.623306,-61.546310,-22.42
76203,66.623306,-61.462978,-22.26
76204,66.623306,-61.379650,-22.18


### Spring historic mean

In [133]:
dftsph = mean_season("spring", dfse45, dfse85)
dftsph

,lat,lon,spring_hist_p50
0,40.041039,-89.045212,11.53
1,40.041039,-88.961884,11.50
2,40.041039,-88.878548,11.48
3,40.041039,-88.795219,11.44
4,40.041039,-88.711891,11.46
...,...,...,...
76201,66.623306,-61.629639,-14.30
76202,66.623306,-61.546310,-14.62
76203,66.623306,-61.462978,-14.39
76204,66.623306,-61.379650,-14.30


### Summer historic mean

In [134]:
dftsuh = mean_season("summer", dfse45, dfse85)
dftsuh

,lat,lon,summer_hist_p50
0,40.041039,-89.045212,23.56
1,40.041039,-88.961884,23.54
2,40.041039,-88.878548,23.54
3,40.041039,-88.795219,23.54
4,40.041039,-88.711891,23.56
...,...,...,...
76201,66.623306,-61.629639,4.41
76202,66.623306,-61.546310,4.08
76203,66.623306,-61.462978,4.28
76204,66.623306,-61.379650,4.35


### Fall historic mean

In [135]:
dftfah = mean_season("fall", dfse45, dfse85)
dftfah

,lat,lon,fall_hist_p50
0,40.041039,-89.045212,12.78
1,40.041039,-88.961884,12.78
2,40.041039,-88.878548,12.78
3,40.041039,-88.795219,12.76
4,40.041039,-88.711891,12.80
...,...,...,...
76201,66.623306,-61.629639,-6.48
76202,66.623306,-61.546310,-6.84
76203,66.623306,-61.462978,-6.58
76204,66.623306,-61.379650,-6.48


# Merge means with the 45 and 85 DF

In [142]:
dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [143]:
dftgseall

,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90,fall_hist_p50,summer_hist_p50,spring_hist_p50,winter_hist_p50
0,40.041039,-89.045212,-0.84,0.95,1.13,3.00,3.29,5.56,12.31,13.66,...,13.61,13.90,15.06,15.65,16.62,17.39,12.78,23.56,11.53,-1.36
1,40.041039,-88.961884,-0.87,0.94,1.11,2.98,3.26,5.54,12.29,13.63,...,13.60,13.89,15.06,15.65,16.63,17.39,12.78,23.54,11.50,-1.38
2,40.041039,-88.878548,-0.89,0.92,1.10,2.97,3.25,5.53,12.27,13.62,...,13.61,13.89,15.07,15.66,16.63,17.39,12.78,23.54,11.48,-1.40
3,40.041039,-88.795219,-0.91,0.91,1.07,2.94,3.22,5.50,12.24,13.59,...,13.59,13.88,15.05,15.64,16.61,17.37,12.76,23.54,11.44,-1.42
4,40.041039,-88.711891,-0.91,0.92,1.08,2.96,3.22,5.51,12.26,13.61,...,13.63,13.91,15.08,15.67,16.64,17.39,12.80,23.56,11.46,-1.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-18.88,-14.48,-15.12,-10.85,-11.99,-6.04,-13.09,-10.96,...,-6.70,-6.48,-4.47,-4.06,-2.62,-2.02,-6.48,4.41,-14.30,-22.26
76202,66.623306,-61.546310,-19.03,-14.64,-15.29,-11.03,-12.17,-6.26,-13.42,-11.29,...,-7.07,-6.85,-4.86,-4.45,-3.02,-2.42,-6.84,4.08,-14.62,-22.42
76203,66.623306,-61.462978,-18.87,-14.49,-15.14,-10.89,-12.01,-6.13,-13.18,-11.05,...,-6.84,-6.62,-4.63,-4.23,-2.79,-2.19,-6.58,4.28,-14.39,-22.26
76204,66.623306,-61.379650,-18.79,-14.42,-15.08,-10.85,-11.96,-6.11,-13.09,-10.96,...,-6.76,-6.54,-4.56,-4.16,-2.73,-2.13,-6.48,4.35,-14.30,-22.18


In [ ]:
dftgseall.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"var"_seasonal.feather")

In [ ]:
#dftgseall = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/tg_mean_annual.feather")

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [ ]:

             "DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
    "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  

In [148]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [149]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [150]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [151]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_tg_mean_seasonal.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file(short+"_tg_mean_seasonal.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

ERROR:fiona._env:6kl_tg_mean_seasonal.json: No such file or directory


6kl


ERROR:fiona._env:6opqr_tg_mean_seasonal.json: No such file or directory


6opqr


ERROR:fiona._env:TG_tg_mean_seasonal.json: No such file or directory
